This notebook is a solution to Advent of Code 2022 Day 19: https://adventofcode.com/2022/day/19

I thought this was particularly interesting because it could be solved with integer optimization, below is my solution using the "pyomo" package.

First, the instructions can be encoded in a matrix to be used programatically. This "Cost Matrix" looks like this:
 
{    \
'ore': {'ore': 3, 'clay': 0, 'obsidian': 0},  
'clay': {'ore': 3, 'clay': 0, 'obsidian': 0},  
'obsidian': {'ore': 2, 'clay': 19, 'obsidian': 0},  
'geode': {'ore': 2, 'clay': 0, 'obsidian': 12}   
}  

 [ 3.  3.  2.  2.]  
 [ 0.  0. 19.  0.]  
 [ 0.  0.  0. 12.]  
 [ 0.  0.  0.  0.]
 
where each row and column represent an input (here it's ore, clay, obsidian, geode in that order), and the number for a given row and column tells how much [row] it takes to produce [column]. For example, the 19 says it takes 19 clay to produce an obsidian.

While solving the problem, we are primarily tracking the number of machines we have per timestep. This has a number of advantages when building our constraints:
 - Machines are our primary costs.
 - Since each machine can produce 1 x per minute, we can easily calculate how many x's we've produced by cumulative-summing our x-machines across time (and then subtracting our final count of x-machines because they don't produce x on the timestep they are built).

This is similarly represented by a matrix with columns being ore, clay, obsidian, geode again and each row representing a timestep. A realistic machine-matrix is shown below:

   [  
   [1., 0., 0., 0.],  
    [1., 0., 0., 0.],  
    [1., 0., 0., 0.],  
    [2., 0., 0., 0.],  
    [2., 0., 0., 0.],  
    [2., 1., 0., 0.],  
    [2., 1., 0., 0.],  
    [2., 2., 0., 0.],  
    [2., 3., 0., 0.],  
    [2., 3., 0., 0.],  
    [2., 4., 0., 0.],  
    [2., 5., 0., 0.],  
    [2., 5., 0., 0.],  
    [2., 6., 0., 0.],  
    [2., 6., 1., 0.],  
    [2., 6., 1., 0.],  
    [2., 7., 1., 0.],  
    [2., 8., 1., 0.],  
    [2., 8., 2., 0.],  
    [2., 8., 3., 0.],  
    [2., 9., 3., 0.],  
    [2., 9., 3., 0.],  
    [2., 9., 3., 1.],  
    [2., 9., 3., 1.]  
    ]  

It starts at time 1, so for example time 6 (row 6) shows that we have 2 ore machines and 1 clay machine

The point of the transformations above is to give structure to the solver. We are going to tell the solver to maximize the number of geodes produced by minute 24. However, doing so without any constraints will result in an unbounded solution - after all the solver doesn't inherently know that we can't build 2^64-1 geode machines in the first timestep. It doesn't even know that we start with 1 ore machine, so it could fill in whatever it wants in the machine matrix!

So we must build some constraints:

 - "initial_state_constraint". Simply ensures that we start with 1 ore machine and nothing else. This is implemented by ensuring that if we are on the first timestep, the ore position must be 1 and the rest must be 0.

 - "net_ore_constraint". We can calculate the total cost of our machines for any given timestep, and we can calculate the total number of various ore's we've gathered. Thus, our solutions are only valid if, for every timestep, our total cost is less than or equal to our total production. Basically, we can't build machines if we don't have the ores necessary.
 
 - "only_machine_per_minute_constraint". The original problem dictates we can only build 1 machine per timestep. This can be solved by summing our machine count for every timestep, and ensuring that between timesteps, the count differs by at most 1.
 
 - "monotonically_increasing_machines_constraint". The original problem only allows us to buy machines. Note that in our net_ore_constraint, we calculate the cost and production of our machines/ores. The issue is that the solver may try to sell machines to fulfill the constraint, e.g. maybe for some timestep it could be optimal to sell a obsidian machine, and buy a ore and a clay machine, and have the net cost still be fulfilled. This constraint avoid that problem. The implementation is similar to the above, we must ensuring that the total number of machines between each timestep either stays the same or increases).
 
Our objective function is maximizing the total number of geodes we produce. With regards to our machine matrix, this is done by summing the total amount of geode machines we have for each timestep (since the machine matrix keeps track of the cumulative number of each machine, and since each machine produces one thing per second) and subtracting our final count of them (since we don't get the results of the last timestep). 

It turns out that this is enough to solve the problem! The solver will try to maximize our objective function subject to our constraints. The results are stored in the model object, and with a little bit of extraction work we can get the final objective function value, as well as the machine matrix. As a bonus, part 2 is just running it for 32 timesteps, which is a trivial input to our model.

I've always found these sorts of solvers interesting because the problem is tenable logically but totally impossible to solve by hand. I've described the logic to setting up the problem above, and the conversion to code is not too complicated, but the actual problem is gargantuan. The net_ore_constraint, for example, can be described logically, but is actually expressed as 96 (24\*4) separate equations that must be satisfied: for each timestep, and for each ore-type, the amount we've produced for that ore-type must equal the amount we've used of that ore-type. There are 215 constraints for the 24 timestep solve, and you could see how the number of constraints could balloon if you had dozens of ores for hundreds of timesteps instead.

But it's always fun to bust out the linear programming. I don't often find a use for it but it's nice to brush up on.


Additional details:
1. Below is an example of one part of the net_ore_constraint: \
\
(5, 0): machine_time_matrix[0,0] + machine_time_matrix[1,0] + machine_time_matrix[2,0] + machine_time_matrix[3,0] + 1 - ((machine_time_matrix[5,0] - machine_time_matrix[0,0])\*4.0 + (machine_time_matrix[5,1] - machine_time_matrix[0,1])\*2.0 + (machine_time_matrix[5,2] - machine_time_matrix[0,2])\*3.0 + (machine_time_matrix[5,3] - machine_time_matrix[0,3])\*2.0))

The indices here are 0-indexed, so (5, 0) translates to timestep 6 and ores.

Our total ore production is [machine_time_matrix[0,0] + machine_time_matrix[1,0] + machine_time_matrix[2,0] + machine_time_matrix[3,0] + 1]. Note that it's an implementation detail that we go up to 3 (timestep 4) only. The problem specifies that it takes 1 timestep to actually build the machine, which is equivalent to using 1 less day's worth of production and instantly building the machine.

Our costs are:\
(machine_time_matrix[5,0] - machine_time_matrix[0,0])\*4.0 + \
(machine_time_matrix[5,1] - machine_time_matrix[0,1])\*2.0 + \
(machine_time_matrix[5,2] - machine_time_matrix[0,2])\*3.0 + \
(machine_time_matrix[5,3] - machine_time_matrix[0,3])\*2.0

Which you can see is just the number of machines we've added since the start, multiplied by their respective ore costs. So net_ore_constraint (5, 0) is ensuring that the ore we've produced up until timestep 6 is sufficient to have covered our total ore spent on machines.

In [34]:
# Set-up + parsing data

inp = """Blueprint 1:
  Each ore robot costs 4 ore.
  Each clay robot costs 2 ore.
  Each obsidian robot costs 3 ore and 14 clay.
  Each geode robot costs 2 ore and 7 obsidian.

Blueprint 2:
  Each ore robot costs 2 ore.
  Each clay robot costs 3 ore.
  Each obsidian robot costs 3 ore and 8 clay.
  Each geode robot costs 3 ore and 12 obsidian."""
inp = inp.split("\n\n")

inp = """Blueprint 1: Each ore robot costs 3 ore. Each clay robot costs 3 ore. Each obsidian robot costs 2 ore and 19 clay. Each geode robot costs 2 ore and 12 obsidian.
Blueprint 2: Each ore robot costs 3 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 19 clay. Each geode robot costs 2 ore and 9 obsidian.
Blueprint 3: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 17 clay. Each geode robot costs 3 ore and 10 obsidian.
Blueprint 4: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 2 ore and 14 clay. Each geode robot costs 4 ore and 15 obsidian.
Blueprint 5: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 2 ore and 12 clay. Each geode robot costs 3 ore and 15 obsidian.
Blueprint 6: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 ore and 8 clay. Each geode robot costs 3 ore and 19 obsidian.
Blueprint 7: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 ore and 7 clay. Each geode robot costs 4 ore and 17 obsidian.
Blueprint 8: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 ore and 14 clay. Each geode robot costs 3 ore and 16 obsidian.
Blueprint 9: Each ore robot costs 2 ore. Each clay robot costs 4 ore. Each obsidian robot costs 2 ore and 16 clay. Each geode robot costs 2 ore and 9 obsidian.
Blueprint 10: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 ore and 5 clay. Each geode robot costs 3 ore and 7 obsidian.
Blueprint 11: Each ore robot costs 2 ore. Each clay robot costs 4 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 4 ore and 9 obsidian.
Blueprint 12: Each ore robot costs 3 ore. Each clay robot costs 4 ore. Each obsidian robot costs 3 ore and 16 clay. Each geode robot costs 3 ore and 14 obsidian.
Blueprint 13: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 18 clay. Each geode robot costs 2 ore and 19 obsidian.
Blueprint 14: Each ore robot costs 3 ore. Each clay robot costs 3 ore. Each obsidian robot costs 2 ore and 13 clay. Each geode robot costs 3 ore and 12 obsidian.
Blueprint 15: Each ore robot costs 3 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 16 clay. Each geode robot costs 3 ore and 9 obsidian.
Blueprint 16: Each ore robot costs 3 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 17 clay. Each geode robot costs 4 ore and 8 obsidian.
Blueprint 17: Each ore robot costs 2 ore. Each clay robot costs 2 ore. Each obsidian robot costs 2 ore and 17 clay. Each geode robot costs 2 ore and 10 obsidian.
Blueprint 18: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 16 clay. Each geode robot costs 2 ore and 11 obsidian.
Blueprint 19: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 ore and 12 clay. Each geode robot costs 3 ore and 8 obsidian.
Blueprint 20: Each ore robot costs 3 ore. Each clay robot costs 4 ore. Each obsidian robot costs 3 ore and 12 clay. Each geode robot costs 3 ore and 17 obsidian.
Blueprint 21: Each ore robot costs 3 ore. Each clay robot costs 3 ore. Each obsidian robot costs 2 ore and 7 clay. Each geode robot costs 2 ore and 9 obsidian.
Blueprint 22: Each ore robot costs 2 ore. Each clay robot costs 4 ore. Each obsidian robot costs 3 ore and 17 clay. Each geode robot costs 4 ore and 20 obsidian.
Blueprint 23: Each ore robot costs 4 ore. Each clay robot costs 3 ore. Each obsidian robot costs 2 ore and 19 clay. Each geode robot costs 3 ore and 10 obsidian.
Blueprint 24: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 3 ore and 9 clay. Each geode robot costs 3 ore and 7 obsidian.
Blueprint 25: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 3 ore and 19 obsidian.
Blueprint 26: Each ore robot costs 3 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 16 clay. Each geode robot costs 3 ore and 20 obsidian.
Blueprint 27: Each ore robot costs 3 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 ore and 19 clay. Each geode robot costs 4 ore and 11 obsidian.
Blueprint 28: Each ore robot costs 4 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 11 clay. Each geode robot costs 4 ore and 7 obsidian.
Blueprint 29: Each ore robot costs 3 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 2 ore and 12 obsidian.
Blueprint 30: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 ore and 8 clay. Each geode robot costs 2 ore and 15 obsidian."""
inp = inp.split('\n')

split = []
for test in inp:
    test = test.replace(':', ':\n  ')
    test = test.replace('.', '.\n  ')
    test = test[:-3]
    split.append(test)
inp = split

import re

def parse_blueprint(blueprint):
    # Dictionary to hold the parsed costs
    robot_costs = {}

    # Split the blueprint string into lines
    lines = blueprint.strip().split('\n')

    # Regular expression to match the lines with cost details
    cost_pattern = re.compile(r'Each (\w+) robot costs (\d+) ore(?: and (\d+) clay)?(?: and (\d+) obsidian)?.')

    # Parse each line
    for line in lines:
        match = cost_pattern.match(line.strip())
        if match:
            # Extract robot type and costs
            robot_type = match.group(1)
            ore_cost = int(match.group(2))
            clay_cost = int(match.group(3)) if match.group(3) else 0
            obsidian_cost = int(match.group(4)) if match.group(4) else 0

            # Add to the dictionary
            robot_costs[robot_type] = {'ore': ore_cost, 'clay': clay_cost, 'obsidian': obsidian_cost}

    return robot_costs

blueprints = []
for blueprint in inp:
    blueprints.append(parse_blueprint(blueprint))

In [35]:
from pyomo.core import *
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition
import pyomo.environ as pe

import numpy as np

import logging
logging.getLogger('pyomo.core').setLevel(logging.INFO)

solver = 'glpk'
opt = SolverFactory(solver)

In [36]:
# Objective function is reducing the sum of all elements in the array
def obj_func(model):
    # Maximize amount of geodes or minimize the negative amount of geodes
    return sum([model.machine_time_matrix[i, 3] for i in model.I]) - model.machine_time_matrix[model.i, 3]


def net_ore_constraint(model, i, jj):
#     if i == 0:
#         return Constraint.Skip
    total_ores = [sum([model.machine_time_matrix[ii, j] for ii in range(i-1)]) for j in model.J]
    total_ores[0] += 1  #  Start at minute 1
#     total_ores[0] += model.blueprint_array[0, 0]
    
    machines_built = [model.machine_time_matrix[i, j] - model.machine_time_matrix[0, j] for j in model.J]
#     machines_built = [model.machine_time_matrix[i, j] for j in model.J]
    costs = [sum([(machines_built[k] * model.blueprint_array[j, k]) for k in model.J]) for j in model.J]
    return total_ores[jj] - costs[jj] >= 0


def only_machine_per_minute_constraint(model, i):
    if i == min(model.I):  # No constraint needed for the first
        return Constraint.Skip
    current_machines = sum([model.machine_time_matrix[i, j] for j in model.J])
    prev_machines = sum([model.machine_time_matrix[i-1, j] for j in model.J])
    
    return current_machines - prev_machines <= 1


def monotonically_increasing_machines_constraint(model, i, j):
    if i == max(model.I):  # No constraint needed for the last row
        return Constraint.Skip
    # Else, ensure that the value in the next row is greater than or equal
    return model.machine_time_matrix[i+1, j] >= model.machine_time_matrix[i, j]


def initial_state_constraint(model, j):
    if j == 0:
        return model.machine_time_matrix[0, j] == 1
    else:
        return model.machine_time_matrix[0, j] == 0
    
def test_constraint(model, j):
    if j == 0:
        return model.machine_time_matrix[2, j] == 1
    elif j == 1:
        return model.machine_time_matrix[2, j] == 1
    else:
        return model.machine_time_matrix[2, j] == 0
    
    
def all_integers_constraint(model, i, j):
    return model.machine_time_matrix[i, j].is_integer()


def array_to_dict_2d(array):
    dct = dict()
    
    for i in range(len(array)):
        for j in range(len(array[0])):
            dct[i, j] = float(array[i, j])
    return dct


def array_to_dict_1d(array):
    dct = dict()
    
    for i in range(len(array)):
        dct[i] = float(array[i])
    return dct


class ModelFactory(object):
    def __init__(self, machine_time_matrix, blueprint_array):
        self.machine_time_matrix = machine_time_matrix
        self.blueprint_array = blueprint_array
        
        self.LENGTH_I, self.LENGTH_J = self.machine_time_matrix.shape
        
    def create_model(self):
        model = ConcreteModel()
        model._blueprint_array = self.blueprint_array

        model.i = Param(within=NonNegativeIntegers, default=self.LENGTH_I-1)
        model.j = Param(within=NonNegativeIntegers, default=self.LENGTH_J-1)

        model.I = RangeSet(0, model.i)
        model.J = RangeSet(0, model.j)
        
        model.blueprint_dim = RangeSet(0, 3)

        model.machine_time_matrix = Var(model.I, model.J, 
                initialize=array_to_dict_2d(self.machine_time_matrix), within=NonNegativeIntegers)
        model.blueprint_array = Param(model.blueprint_dim, model.blueprint_dim, 
                initialize=array_to_dict_2d(self.blueprint_array), mutable=False)
        
        # OBJECTIVE + CONSTRAINTS
        model.obj = Objective(rule=obj_func, sense=maximize)
        model.monotonically_increasing_machines_constraint = Constraint(model.I, model.J, rule=monotonically_increasing_machines_constraint)
        model.net_ore_constraint = Constraint(model.I, model.J, rule = net_ore_constraint)
        model.initial_state_constraint = Constraint(model.J, rule = initial_state_constraint)
        model.only_machine_per_minute_constraint = Constraint(model.I, rule = only_machine_per_minute_constraint)
        
#         model.test_constraint = Constraint(model.J, rule = test_constraint)
        return model

In [37]:
def solve_for_blueprint(blueprint, time, debug=False):
    machine_time_matrix = np.zeros((time, 4))
    machine_time_matrix[:, 0] = 1

    blueprint_array = np.zeros([4, 4])
    bp = blueprint

    mapping = {'ore': 0, 'clay': 1, 'obsidian': 2, 'geode': 3}
    for ore_col, cost_dict in bp.items():
        for ore_row, cost in cost_dict.items():
            blueprint_array[mapping[ore_row], mapping[ore_col]] = cost
    if debug:
        print(blueprint)
        print("Blueprint Cost Array:")
        print(blueprint_array)
        
    mf = ModelFactory(machine_time_matrix, blueprint_array)
    m = mf.create_model()
    result = opt.solve(m, tee=False)
    
    if debug:
        return m
    return value(m.obj)

import pandas as pd

def get_costs_and_prods(display, i):
    machines_built = display.loc[i] - display.loc[0]
    costs = (machines_built[1:].values * m._blueprint_array).sum(1)
    produced = display.values[:i, 1:].sum(0)
    
    return costs, produced


def get_display(m):
    display = np.zeros((m.i()+1, m.j()+2))
    for i in range(m.i()+1):
        for j in range(m.j()+2):
            if j == 0:
                display[i, j] = i+1
            else:
                display[i, j] = value(m.machine_time_matrix[i, j-1])
                
    display = pd.DataFrame(display, dtype=int)
#     display.columns = ["Time", "Ore", "Clay", "Obsidian", "Geode"]
    
    costs_prods = []
    for i in range(len(display)):
        costs, prods = get_costs_and_prods(display, i)
        costs_prods.append(np.concatenate([costs, prods]))

    cost_prod_array = np.vstack(costs_prods)
        
    df = pd.concat([display, pd.DataFrame(np.vstack(cost_prod_array))], axis=1)
    df.columns = ["Time", 
                  "Ore-Mach", "Clay-Mach", "Obsidian-Mach", "Geode-Mach", 
                  "Ore-Cost", "Clay-Cost", "Obsidian-Cost", "Geode-Cost",
                  "Ore-Prod", "Clay-Prod", "Obsidian-Prod", "Geode-Prod"]

    df = df.astype(int)
    
    df.loc[:, "Ore-Prod"] = df.loc[:, "Ore-Prod"] + 1
        
    return df


In [38]:
# m = solve_for_blueprint(blueprints[0], 24, True)
# m.net_ore_constraint.pprint()
# display = get_display(m)
# m.obj()
# display
# display.loc[:, ['Ore-Prod', 'Clay-Prod', 'Obsidian-Prod', 'Geode-Prod']].values - display.loc[:, ['Ore-Cost', 'Clay-Cost', 'Obsidian-Cost', 'Geode-Cost']].values

In [39]:
quality_level = []
for i, bp in enumerate(blueprints):
    quality_level.append((i+1) * solve_for_blueprint(bp, 24))

In [40]:
# quality_level

In [41]:
sum(quality_level)

1589.0

In [42]:
## PART 2 ##

In [43]:
values = []
for i, bp in enumerate(blueprints[:3]):
    values.append(solve_for_blueprint(bp, 32))

In [44]:
values

[22.0, 29.0, 46.0]

In [45]:
import operator
from functools import reduce
reduce(operator.mul, values)

29348.0